In [31]:
# Importing modules
import pandas as pd
import os

# Read data into papers
data = pd.read_csv('./data1.7.csv')
# Print head
data.head()

,Place,Details,Class
0,Signal Hill,"['signal', 'hill', 'located', 'side', 'table',...",None Related
1,Alexandria Library,"['futuristic', 'building', 'reading', 'place',...",None Related
2,Pompey's Pillar,"['pompeii', 'column', 'towers', 'ruins', 'temp...",None Related
3,Nairobi National Park,"['nairobi', 'national', 'park', '8', 'kilomete...",None Related
4,Giraffe Centre,"['giraffe', 'center', 'home', 'endangered', 'r...",None Related


In [32]:
# Load the regular expression library
import re
# Remove punctuation
data['Details'] = \
data['Details'].map(lambda x: re.sub('[,\.!?]', '', x))
# Convert the titles to lowercase
data['Details'] = \
data['Details'].map(lambda x: x.lower())
# Print out the first rows of papers
data['Details'].head()

<>:5: DeprecationWarning: invalid escape sequence '\.'
<>:5: DeprecationWarning: invalid escape sequence '\.'
C:\Users\ASUS TUF\AppData\Local\Temp\ipykernel_7424\617571986.py:5: DeprecationWarning: invalid escape sequence '\.'
  data['Details'].map(lambda x: re.sub('[,\.!?]', '', x))


0    ['signal' 'hill' 'located' 'side' 'table' 'mou...
1    ['futuristic' 'building' 'reading' 'place' 'se...
2    ['pompeii' 'column' 'towers' 'ruins' 'temple' ...
3    ['nairobi' 'national' 'park' '8' 'kilometers' ...
4    ['giraffe' 'center' 'home' 'endangered' 'roths...
Name: Details, dtype: object

In [33]:
import gensim
from gensim.utils import simple_preprocess
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])
def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]
data = data.Details.values.tolist()
data_words = list(sent_to_words(data))
# remove stop words
data_words = remove_stopwords(data_words)
print(data_words[:1][0][:30])

[nltk_data] Downloading package stopwords to C:\Users\ASUS
[nltk_data]     TUF\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['signal', 'hill', 'located', 'side', 'table', 'mountain', 'named', 'firing', 'cannons', 'noon', 'end', 'also', 'known', 'lion', 'head', 'mountain', 'appearance', 'lion', 'head', 'signal', 'hill', 'meters', 'sea', 'level', 'multiple', 'angles', 'mountain', 'overlook', 'cityscape', 'cape', 'town', 'vast', 'atlantic', 'ocean', 'panoramic', 'view', 'famous', 'table', 'mountain', 'signal', 'hill', 'also', 'good', 'place', 'watch', 'cape', 'town', 'night', 'view', 'known', 'one', 'three', 'major', 'night', 'views', 'world', 'wonderful', 'views', 'multiple', 'angles', 'mountain', 'dazzling', 'crystal', 'clear', 'endless', 'fire', 'trees', 'silver', 'flowers', 'stretch', 'dozens', 'kilometers', 'breathtaking', 'unforgettable', 'traditional', 'ceremony', 'held', 'every', 'day', 'cape', 'town', 'noon', 'except', 'sundays', 'cannon', 'salvo', 'signal', 'hill', 'punctual', 'south', 'africans', 'even', 'used', 'set', 'clocks']


In [34]:
import gensim.corpora as corpora
# Create Dictionary
id2word = corpora.Dictionary(data_words)
# Create Corpus
texts = data_words
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
# View
print(corpus[:1][0][:30])

[(0, 1), (1, 2), (2, 2), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 3), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 2), (28, 1), (29, 4)]


In [35]:
from pprint import pprint
# number of topics
num_topics = 2
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.009*"also" + 0.005*"located" + 0.005*"park" + 0.004*"one" + 0.004*"many" '
  '+ 0.004*"lake" + 0.004*"museum" + 0.004*"world" + 0.004*"city" + '
  '0.004*"see"'),
 (1,
  '0.008*"also" + 0.004*"located" + 0.004*"many" + 0.004*"island" + '
  '0.004*"lake" + 0.004*"one" + 0.004*"built" + 0.004*"church" + 0.003*"water" '
  '+ 0.003*"sea"')]


In [36]:
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis

vis_data = gensimvis.prepare(lda_model, corpus, id2word)
pyLDAvis.display(vis_data)
